In [167]:
import pandas as pd
import numpy as np

In [168]:
train_data = pd.read_csv("../../train.csv")
test_data = pd.read_csv("../../test.csv")

In [169]:
#Removing cols

#removing 'cabin' column because it has many NAN cols
cols = ['Ticket', 'Name', 'Cabin', 'Ticket', 'PassengerId']
train_data.drop(columns=cols, inplace=True)
test_data.drop(columns=cols, inplace=True)

In [170]:
train_data.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [171]:
train_data.isnull().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Embarked     True
dtype: bool

In [172]:
#fill NAN
def replace_numeric(columns: list, dataset: pd.DataFrame):
    nan_list = dataset.isnull().any()
    for col in columns:
        if nan_list[col]:
            temp = dataset[col]
            temp = temp.dropna()
            dataset[col].fillna(temp.mean(), inplace=True)

def replace_categorical(columns: list, dataset: pd.DataFrame):
    nan_list = dataset.isnull().any()
    for col in columns:
        if nan_list[col]:
            temp = dataset[col]
            temp = temp.dropna()
            dataset[col].fillna(value=temp.mode()[0], inplace=True)

numeric_cols = ['Age', 'Fare', 'Parch', 'SibSp']
replace_numeric(numeric_cols, train_data)
replace_numeric(numeric_cols, test_data)

categorical_cols = ['Embarked', 'Sex']
replace_categorical(categorical_cols, train_data)
replace_categorical(categorical_cols, test_data)

In [174]:
train_data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [175]:
test_data.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64